In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abs-dataset/dataset_final.csv


In [2]:
train=pd.read_csv('/kaggle/input/abs-dataset/dataset_final.csv')

In [3]:
texts=train['Text'].tolist()
import pandas as pd
from tokenizers import ByteLevelBPETokenizer
import os


# Extract text column (Replace 'text_column' with the actual column name)
text_data = train["Text"].dropna().tolist()

# Save as a single text file for training the tokenizer
with open("dataset.txt", "w", encoding="utf-8") as f:
    for line in text_data:
        f.write(line + "\n")  # Write each text entry as a new line

# Initialize tokenizer
tokenizer = ByteLevelBPETokenizer()

# Train the tokenizer on the extracted text
tokenizer.train(files=["dataset.txt"], vocab_size=52000, min_frequency=2)
os.makedirs('.tokenizer_gpt',exist_ok=True)

# Save the tokenizer model
tokenizer.save_model(".tokenizer_gpt")

['.tokenizer_gpt/vocab.json', '.tokenizer_gpt/merges.txt']

In [4]:
# Save the tokenizer vocabulary (vocab.json) and merges.txt
tokenizer.save(".tokenizer_gpt/vocab.json", pretty=True) # Save only vocab.json
tokenizer.save(".tokenizer_gpt/merges.txt")

In [5]:
from transformers import PreTrainedTokenizerFast
tokenizer_file_path = os.path.abspath(".tokenizer_gpt/vocab.json")

tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=tokenizer_file_path 
)

# Add padding token if not present
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

1

In [6]:
import transformers
transformers.__version__

'4.51.1'

In [7]:
!pip install --no-cache-dir   bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
from transformers import BitsAndBytesConfig
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
     bnb_4bit_use_double_quant=True
)

In [9]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("mistralai/Mistral-7B-v0.1",token="",quantization_config=bnb_config)


2025-04-29 07:41:57.346685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745912517.369097     106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745912517.375862     106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(model)

MistralForSequenceClassification(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
  

In [11]:
for param in model.model.parameters():
    param.require_grad=False
for param in model.score.parameters():
    param.require_grad=True

In [12]:
from peft import LoraConfig,get_peft_model
lora=LoraConfig(
    r=4,
    lora_alpha=4,
    lora_dropout=0.2
)
model=get_peft_model(model,lora)

In [13]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [14]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    train,
    test_size=0.2,
    random_state=42
)
test, valid = train_test_split(
    test,
    test_size=0.5,
    random_state=42
)

In [15]:
def tokenize_texts(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt",padding_side='right')

# Create PyTorch Dataset
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenize_texts(texts)
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

In [16]:
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
train_dataset = TextDataset(train['Text'].tolist(), train['Class'].tolist())
val_dataset = TextDataset(test['Text'].tolist(),test['Class'].tolist())

In [17]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
num_training_steps = len(train_loader) * 3  # 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
EPOCHS = 1

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for batch in train_loader:
        batch = {key: val.to(device) for key, val in batch.items()}

        optimizer.zero_grad()
        batch.pop('token_type_ids',None)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [ ]:
model.eval()
correct = 0
total = 0
pred_label=[]
with torch.no_grad():
    for batch in val_loader:
        batch = {key: val.to(device) for key, val in batch.items()}
        batch.pop('token_type_ids',None)
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        pred_label.append(predictions)
        correct += (predictions == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

accuracy = correct / total
print(f"Validation Accuracy: {accuracy:.4f}")

In [ ]:
print(pred_label)

In [ ]:
model.save_pretrained("./trbt-tokenizer_mdel")
tokenizer.save_pretrained("./trbt-tokenizer_model")